In [5]:
from pymarc import MARCReader, Field
from rdflib import Graph, Namespace, Literal

import sys
import logging

SKOS=Namespace('http://www.w3.org/2004/02/skos/core#')
YSO=Namespace('http://www.yso.fi/onto/yso/')

# initialize logging
logformat = '%(levelname)s: %(message)s'
loglevel = logging.DEBUG
logging.basicConfig(format=logformat, level=loglevel)

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")

DEBUG: test


In [2]:
sys.argv = ['ysa2yso', 
            'marc.exp.20180125.1556.1',
            '/home/oisuomin/git/Finto-data/vocabularies/ysa/ysa-skos.ttl',
            '/home/oisuomin/git/Finto-data/vocabularies/yso/yso-skos.ttl',
            '/home/oisuomin/git/Finto-data/vocabularies/yso-paikat/yso-paikat-skos.ttl']

# load YSA
logging.info("Loading YSA from %s ...", sys.argv[2])
ysa = Graph()
ysa.parse(sys.argv[2], format='turtle')

# load YSO
logging.info("Loading YSO from %s ...", sys.argv[3])
yso = Graph()
yso.parse(sys.argv[3], format='turtle')

# load YSO Places
logging.info("Loading YSO Places from %s ...", sys.argv[4])
ysop = Graph()
ysop.parse(sys.argv[4], format='turtle')


INFO: Loading YSA from /home/oisuomin/git/Finto-data/vocabularies/ysa/ysa-skos.ttl ...
INFO: Loading YSO from /home/oisuomin/git/Finto-data/vocabularies/yso/yso-skos.ttl ...
INFO: Loading YSO Places from /home/oisuomin/git/Finto-data/vocabularies/yso-paikat/yso-paikat-skos.ttl ...


<Graph identifier=N0a9f6df2f5f94724b30bc9423b74485d (<class 'rdflib.graph.Graph'>)>

In [25]:


def combined_label(f):
    labels = f.get_subfields('a', 'x', 'z')
    label = ' -- '.join(labels)
    return label

def ysalabel_to_ysauri(label):
    # prefLabel
    uri = ysa.value(None, SKOS.prefLabel, Literal(label, "fi"))
    if uri is not None:
        return uri
    # altLabel
    uri = ysa.value(None, SKOS.altLabel, Literal(label, "fi"))
    if uri is not None:
        return uri
    return None

def ysauri_to_ysouris(ysauri):
    if ysauri is None:
        return []
    return [uri for prop in (SKOS.closeMatch, SKOS.exactMatch)
                for uri in ysa.objects(ysauri, prop)
            if uri.startswith(YSO)]

def ysouri_label_ft(uri):
    for voc, ft in ((yso, '650'), (ysop, '651')):
        labels = voc.preferredLabel(uri, lang='fi')
        if len(labels) > 0:
            return (labels[0][1], ft)
    return (None, None)

with open(sys.argv[1], 'rb') as fh:
    reader = MARCReader(fh)
    count = 0
    for rec in reader:
        changed_fields = set()
        recid = rec['001'].value()
        
        for ftype in ('650','651'):
            for subjf in rec.get_fields(ftype):
                if '2' in subjf and subjf['2'] == 'ysa':
                    label = combined_label(subjf)
                    ysauri = ysalabel_to_ysauri(label)
                    logging.debug('%s %s: %s\t<%s>', recid, ftype, label, ysauri)
                    if ysauri is None:
                        continue
                    ysouris = ysauri_to_ysouris(ysauri)
                    if len(ysouris) == 0:
                        continue
                    rec.remove_field(subjf)
                    changed_fields.add(ftype)
                    for ysouri in ysouris:
                        ysolabel, ysoft = ysouri_label_ft(ysouri)
                        if ysolabel is None:
                            logging.warning("No label found for YSO URI <%s>, skipping" % ysouri)
                            continue
                        if ysoft != ftype:
                            logging.warning("Field type mismatch: %s != %s", ftype, ysoft)
                        logging.debug("\t\t\t<%s>\t'%s' %s", ysouri, ysolabel, ysoft)
                        fld = Field(tag=ysoft,
                                    indicators=[' ','7'],
                                    subfields=[
                                        'a', ysolabel,
                                        '0', str(ysouri),
                                        '2', 'yso/fin'
                                    ])
                        rec.add_ordered_field(fld)
        for tag in changed_fields:
            logging.info('Sorting field %s', tag)
            fields = rec.get_fields(tag)
            rec.remove_fields(tag)
            fields.sort(key=lambda f:f.format_field())
            for f in fields:
                logging.info('field value: %s', f.format_field())
                rec.add_ordered_field(f)

#        sys.stdout.write(rec.as_marc())
        count += 1
        
        if count >= 50:
            break

logging.info("Converted %d records", count)

DEBUG: 1129319 650: kansanmusiikki	<http://www.yso.fi/onto/ysa/Y95864>
DEBUG: 			<http://www.yso.fi/onto/yso/p2841>	'kansanmusiikki' 650
DEBUG: 1129319 650: opetus	<http://www.yso.fi/onto/ysa/Y97807>
DEBUG: 			<http://www.yso.fi/onto/yso/p2630>	'opetus' 650
DEBUG: 1129319 650: perusopetus	<http://www.yso.fi/onto/ysa/Y107816>
DEBUG: 			<http://www.yso.fi/onto/yso/p19327>	'perusopetus' 650
DEBUG: 1129319 650: yläkoulu	<http://www.yso.fi/onto/ysa/Y159125>
DEBUG: 			<http://www.yso.fi/onto/yso/p21292>	'yläkoulu' 650
INFO: Sorting field 650
INFO: field value: kansanmusiikki http://www.yso.fi/onto/yso/p2841 yso/fin
INFO: field value: opetus http://www.yso.fi/onto/yso/p2630 yso/fin
INFO: field value: perusopetus http://www.yso.fi/onto/yso/p19327 yso/fin
INFO: field value: yläkoulu http://www.yso.fi/onto/yso/p21292 yso/fin
DEBUG: 1129320 650: ympäristönsuojelu	<http://www.yso.fi/onto/ysa/Y100453>
DEBUG: 			<http://www.yso.fi/onto/yso/p13>	'ympäristönsuojelu' 650
DEBUG: 1129320 650: ympäristönh

INFO: Sorting field 651
INFO: field value: Etelä-Savo http://www.yso.fi/onto/yso/p105096 yso/fin
INFO: Sorting field 650
INFO: field value: alueidenkäyttö http://www.yso.fi/onto/yso/p6753 yso/fin
INFO: field value: kaavoitus http://www.yso.fi/onto/yso/p8268 yso/fin
INFO: field value: maakunnat http://www.yso.fi/onto/yso/p6819 yso/fin
INFO: field value: maakuntakaavoitus http://www.yso.fi/onto/yso/p12274 yso/fin
DEBUG: 1129332 651: Latvia	<http://www.yso.fi/onto/ysa/Y94259>
DEBUG: 			<http://www.yso.fi/onto/yso/p94259>	'Latvia' 651
DEBUG: 1129332 651: Liettua	<http://www.yso.fi/onto/ysa/Y94268>
DEBUG: 			<http://www.yso.fi/onto/yso/p94268>	'Liettua' 651
DEBUG: 1129332 651: Viro	<http://www.yso.fi/onto/ysa/Y104987>
DEBUG: 			<http://www.yso.fi/onto/yso/p104987>	'Viro' 651
DEBUG: 1129332 651: Suomi	<http://www.yso.fi/onto/ysa/Y94426>
DEBUG: 			<http://www.yso.fi/onto/yso/p94426>	'Suomi' 651
INFO: Sorting field 651
INFO: field value: Latvia http://www.yso.fi/onto/yso/p94259 yso/fin
INFO: f

INFO: field value: maatalousalueet http://www.yso.fi/onto/yso/p13471 yso/fin
INFO: field value: maatalousympäristö http://www.yso.fi/onto/yso/p10843 yso/fin
INFO: field value: maisemanhoito http://www.yso.fi/onto/yso/p4575 yso/fin
INFO: field value: perinnemaisema http://www.yso.fi/onto/yso/p2326 yso/fin
INFO: field value: suojavyöhykkeet http://www.yso.fi/onto/yso/p13603 yso/fin
INFO: field value: vesiensuojelu http://www.yso.fi/onto/yso/p3163 yso/fin
DEBUG: 1129342 650: lastenkulttuuri	<http://www.yso.fi/onto/ysa/Y102790>
DEBUG: 			<http://www.yso.fi/onto/yso/p4804>	'lastenkulttuuri' 650
DEBUG: 1129342 650: kulttuuripalvelut	<http://www.yso.fi/onto/ysa/Y96500>
DEBUG: 			<http://www.yso.fi/onto/yso/p16530>	'kulttuuripalvelut' 650
DEBUG: 1129342 650: kulttuuritoiminta	<http://www.yso.fi/onto/ysa/Y121993>
DEBUG: 			<http://www.yso.fi/onto/yso/p4024>	'kulttuuritoiminta' 650
DEBUG: 1129342 650: saavutettavuus	<http://www.yso.fi/onto/ysa/Y134677>
DEBUG: 			<http://www.yso.fi/onto/yso/p858>

DEBUG: 			<http://www.yso.fi/onto/yso/p1210>	'tiet' 650
DEBUG: 1129348 650: valtatiet	<http://www.yso.fi/onto/ysa/Y202706>
DEBUG: 			<http://www.yso.fi/onto/yso/p28219>	'valtatiet' 650
DEBUG: 1129348 650: liikenne	<http://www.yso.fi/onto/ysa/Y96879>
DEBUG: 			<http://www.yso.fi/onto/yso/p3466>	'liikenne' 650
DEBUG: 1129348 650: tieliikenne	<http://www.yso.fi/onto/ysa/Y99486>
DEBUG: 			<http://www.yso.fi/onto/yso/p8934>	'tieliikenne' 650
DEBUG: 1129348 650: kaistat	<http://www.yso.fi/onto/ysa/Y130112>
DEBUG: 			<http://www.yso.fi/onto/yso/p2497>	'kaistat' 650
DEBUG: 1129348 650: tieverkot	<http://www.yso.fi/onto/ysa/Y99538>
DEBUG: 			<http://www.yso.fi/onto/yso/p11246>	'tieverkot' 650
DEBUG: 1129348 651: Valtatie 10	<http://www.yso.fi/onto/ysa/Y192348>
DEBUG: 			<http://www.yso.fi/onto/yso/p26743>	'Valtatie 10' 650
DEBUG: 			<http://www.yso.fi/onto/yso/p192348>	'Valtatie 10' 651
DEBUG: 1129348 651: Tuulos	<http://www.yso.fi/onto/ysa/Y94451>
DEBUG: 			<http://www.yso.fi/onto/yso/p94451>	

DEBUG: 			<http://www.yso.fi/onto/yso/p1583>	'museopedagogiikka' 650
INFO: Sorting field 650
INFO: field value: kokemuksellisuus http://www.yso.fi/onto/yso/p20871 yso/fin
INFO: field value: lapset ysa
INFO: field value: leikki (toiminta) http://www.yso.fi/onto/yso/p316 yso/fin
INFO: field value: luovuuskasvatus http://www.yso.fi/onto/yso/p12886 yso/fin
INFO: field value: museopedagogiikka http://www.yso.fi/onto/yso/p1583 yso/fin
INFO: field value: taidekasvatus http://www.yso.fi/onto/yso/p5457 yso/fin
INFO: field value: työpajat http://www.yso.fi/onto/yso/p19245 yso/fin
INFO: field value: vanhemmat http://www.yso.fi/onto/yso/p4074 yso/fin
INFO: field value: varhainen vuorovaikutus http://www.yso.fi/onto/yso/p11494 yso/fin
INFO: field value: vauvat http://www.yso.fi/onto/yso/p15937 yso/fin
INFO: field value: vuorovaikutus http://www.yso.fi/onto/yso/p10591 yso/fin
INFO: field value: värit (aine) http://www.yso.fi/onto/yso/p2284 yso/fin
DEBUG: 1129353 650: vesihuolto	<http://www.yso.fi/on

DEBUG: 1129357 650: tukipalvelut	<http://www.yso.fi/onto/ysa/Y99614>
DEBUG: 			<http://www.yso.fi/onto/yso/p5118>	'tukipalvelut' 650
DEBUG: 1129357 650: kehittäminen	<http://www.yso.fi/onto/ysa/Y102331>
DEBUG: 			<http://www.yso.fi/onto/yso/p4230>	'kehittäminen' 650
DEBUG: 1129357 650: hankkeet	<http://www.yso.fi/onto/ysa/Y117429>
DEBUG: 			<http://www.yso.fi/onto/yso/p5408>	'hankkeet' 650
DEBUG: 1129357 651: Etelä-Savo	<http://www.yso.fi/onto/ysa/Y105096>
DEBUG: 			<http://www.yso.fi/onto/yso/p105096>	'Etelä-Savo' 651
INFO: Sorting field 651
INFO: field value: Etelä-Savo http://www.yso.fi/onto/yso/p105096 yso/fin
INFO: Sorting field 650
INFO: field value: hankkeet http://www.yso.fi/onto/yso/p5408 yso/fin
INFO: field value: kehittäminen http://www.yso.fi/onto/yso/p4230 yso/fin
INFO: field value: moniammatillisuus http://www.yso.fi/onto/yso/p943 yso/fin
INFO: field value: nuoret http://www.yso.fi/onto/yso/p11617 yso/fin
INFO: field value: ohjaus (neuvominen) http://www.yso.fi/onto/yso/p

DEBUG: 1129360 650: sähkönjohtavuus	<http://www.yso.fi/onto/ysa/Y104478>
DEBUG: 			<http://www.yso.fi/onto/yso/p9399>	'sähkönjohtavuus' 650
INFO: Sorting field 650
INFO: field value: happi http://www.yso.fi/onto/yso/p11518 yso/fin
INFO: field value: laboratoriot http://www.yso.fi/onto/yso/p8598 yso/fin
INFO: field value: laboratoriotutkimus http://www.yso.fi/onto/yso/p6757 yso/fin
INFO: field value: luotettavuus http://www.yso.fi/onto/yso/p1629 yso/fin
INFO: field value: lämpötila http://www.yso.fi/onto/yso/p2100 yso/fin
INFO: field value: pH http://www.yso.fi/onto/yso/p4555 yso/fin
INFO: field value: pätevyys http://www.yso.fi/onto/yso/p9363 yso/fin
INFO: field value: sähkönjohtavuus http://www.yso.fi/onto/yso/p9399 yso/fin
INFO: field value: vesi http://www.yso.fi/onto/yso/p3792 yso/fin
INFO: field value: vesianalyysi http://www.yso.fi/onto/yso/p9185 yso/fin
DEBUG: 1129361 650: ympäristövaikutusten arviointi	<http://www.yso.fi/onto/ysa/Y203852>
DEBUG: 			<http://www.yso.fi/onto/yso/p

DEBUG: 1129365 650: hedelmällisyys	<http://www.yso.fi/onto/ysa/Y95311>
DEBUG: 			<http://www.yso.fi/onto/yso/p19514>	'hedelmällisyys' 650
DEBUG: 1129365 650: kehityshäiriöt	<http://www.yso.fi/onto/ysa/Y96041>
DEBUG: 			<http://www.yso.fi/onto/yso/p49>	'kehityshäiriöt' 650
DEBUG: 1129365 650: syöpätaudit	<http://www.yso.fi/onto/ysa/Y99138>
DEBUG: 			<http://www.yso.fi/onto/yso/p678>	'syöpätaudit' 650
INFO: Sorting field 650
INFO: field value: arviointi http://www.yso.fi/onto/yso/p7413 yso/fin
INFO: field value: hedelmällisyys http://www.yso.fi/onto/yso/p19514 yso/fin
INFO: field value: ihotaudit http://www.yso.fi/onto/yso/p8746 yso/fin
INFO: field value: ihottumat http://www.yso.fi/onto/yso/p7582 yso/fin
INFO: field value: kehityshäiriöt http://www.yso.fi/onto/yso/p49 yso/fin
INFO: field value: kemikaalit http://www.yso.fi/onto/yso/p7757 yso/fin
INFO: field value: munuaistaudit http://www.yso.fi/onto/yso/p9826 yso/fin
INFO: field value: munuaisten vajaatoiminta http://www.yso.fi/onto/ys